#### Yu Han, Nov 2022.
We used logistic regression to classify astrocyte cells with deletion vs. w/o deletion. Feature importance score is printed. 

mann-whitney-u-test was adopted to run feature by feature analysis between the deletion group and the control group to find significant features.

Combining morph features and branching features.

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import math
import statistics
import random
import scipy.stats as ss

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit

import warnings
warnings.filterwarnings("ignore")
from pycytominer import aggregate

### logistic regression

In [2]:
#read morph feature profile data of two plates 
df_p1=pd.read_csv('../2019_05_28_Neuronal_Cell_Painting/profiles/2022-08-07_batch/Plate1_PE_PP96/Plate1_PE_PP96_normalized_feature_select_batch.csv.gz')
df_p2=pd.read_csv('../2019_05_28_Neuronal_Cell_Painting/profiles/2022-08-07_batch/PE_PP_Plate2/PE_PP_Plate2_normalized_feature_select_batch.csv.gz')
df_astro=pd.concat([df_p1,df_p2])
df_astro=df_astro.drop(['Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel'], axis=1)
df_astro.head(2)

,Metadata_plate_map_name,Metadata_METADATA_SAMPLE_ID,Metadata_METADATA_DENSITY,Metadata_METADATA_CONDITION,Metadata_METADATA_SOURCE,Metadata_Plate,Metadata_Well,Metadata_Site_Count,Metadata_Count_Cells,Metadata_Count_Cytoplasm,...,Nuclei_Texture_Correlation_RNA_20_02,Nuclei_Texture_Correlation_RNA_20_03,Nuclei_Texture_DifferenceVariance_RNA_10_01,Nuclei_Texture_InfoMeas1_AGP_20_03,Nuclei_Texture_InfoMeas1_AGP_5_00,Nuclei_Texture_InfoMeas1_Brightfield_5_00,Nuclei_Texture_InfoMeas1_DNA_5_02,Nuclei_Texture_InfoMeas1_RNA_5_00,Nuclei_Texture_InfoMeas2_DNA_5_02,Nuclei_Texture_SumVariance_Brightfield_20_00
0,2022-08-07_batch_METADATA,1,5000,CONTROL,HUMAN,Plate1_PE_PP96,A01,7,41,41,...,1.1747,2.27630,-0.18264,0.9503,0.54613,-1.6775,-0.081452,-1.6280,-0.74218,0.77555
1,2022-08-07_batch_METADATA,30,5000,DELETION,HUMAN,Plate1_PE_PP96,A02,7,53,53,...,2.7995,0.17215,-1.34580,-1.3945,-0.47536,-2.9766,0.377130,-1.0986,0.31453,3.22570


In [3]:
#read morph feature profile data of two plates 
df_p1_aug=pd.read_csv('../2019_05_28_Neuronal_Cell_Painting/profiles/2022-08-07_batch/Plate1_PE_PP96/Plate1_PE_PP96_augmented.csv.gz')
df_p2_aug=pd.read_csv('../2019_05_28_Neuronal_Cell_Painting/profiles/2022-08-07_batch/PE_PP_Plate2/PE_PP_Plate2_augmented.csv.gz')
df_astro_aug=pd.concat([df_p1_aug,df_p2_aug])
df_astro_aug.head(2)

,Metadata_plate_map_name,Metadata_METADATA_SAMPLE_ID,Metadata_METADATA_DENSITY,Metadata_METADATA_CONDITION,Metadata_METADATA_SOURCE,Metadata_Plate,Metadata_Well,Metadata_Site_Count,Metadata_Count_Cells,Metadata_Count_Cytoplasm,...,Nuclei_Texture_Variance_RNA_10_02,Nuclei_Texture_Variance_RNA_10_03,Nuclei_Texture_Variance_RNA_20_00,Nuclei_Texture_Variance_RNA_20_01,Nuclei_Texture_Variance_RNA_20_02,Nuclei_Texture_Variance_RNA_20_03,Nuclei_Texture_Variance_RNA_5_00,Nuclei_Texture_Variance_RNA_5_01,Nuclei_Texture_Variance_RNA_5_02,Nuclei_Texture_Variance_RNA_5_03
0,2022-08-07_batch_METADATA,1,5000,CONTROL,HUMAN,Plate1_PE_PP96,A01,7,41,41,...,20.020,20.147,20.796,20.820,20.755,20.861,20.389,20.005,20.606,20.249
1,2022-08-07_batch_METADATA,30,5000,DELETION,HUMAN,Plate1_PE_PP96,A02,7,53,53,...,41.338,41.518,43.668,48.979,43.388,46.283,42.868,42.010,42.858,42.162


In [4]:
#find branching features from the augmented profile
df_branch=df_astro_aug.filter(regex='ObjectSkeleton|Metadata')
df_branch.head(2)

,Metadata_plate_map_name,Metadata_METADATA_SAMPLE_ID,Metadata_METADATA_DENSITY,Metadata_METADATA_CONDITION,Metadata_METADATA_SOURCE,Metadata_Plate,Metadata_Well,Metadata_Site_Count,Metadata_Count_Cells,Metadata_Count_Cytoplasm,Metadata_Count_Nuclei,Metadata_Object_Count,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel
0,2022-08-07_batch_METADATA,1,5000,CONTROL,HUMAN,Plate1_PE_PP96,A01,7,41,41,41,41,43.195,43.683,3.0244,3366.9
1,2022-08-07_batch_METADATA,30,5000,DELETION,HUMAN,Plate1_PE_PP96,A02,7,53,53,53,53,26.604,24.151,3.2075,1390.0


In [5]:
#merge morph and branch profile
df_merge=pd.merge(df_astro, df_branch, on =["Metadata_Plate",'Metadata_Well'])
df_merge.head(2)

,Metadata_plate_map_name_x,Metadata_METADATA_SAMPLE_ID_x,Metadata_METADATA_DENSITY_x,Metadata_METADATA_CONDITION_x,Metadata_METADATA_SOURCE_x,Metadata_Plate,Metadata_Well,Metadata_Site_Count_x,Metadata_Count_Cells_x,Metadata_Count_Cytoplasm_x,...,Metadata_METADATA_SOURCE_y,Metadata_Site_Count_y,Metadata_Count_Cells_y,Metadata_Count_Cytoplasm_y,Metadata_Count_Nuclei_y,Metadata_Object_Count_y,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel
0,2022-08-07_batch_METADATA,1,5000,CONTROL,HUMAN,Plate1_PE_PP96,A01,7,41,41,...,HUMAN,7,41,41,41,41,43.195,43.683,3.0244,3366.9
1,2022-08-07_batch_METADATA,30,5000,DELETION,HUMAN,Plate1_PE_PP96,A02,7,53,53,...,HUMAN,7,53,53,53,53,26.604,24.151,3.2075,1390.0


In [6]:
#check branching features
df_merge_branch = df_merge.filter(regex='ObjectSkeleton')
df_merge_branch.head(2)

,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel
0,43.195,43.683,3.0244,3366.9
1,26.604,24.151,3.2075,1390.0


In [7]:
#code deletion as 1 and control as 0. 
df_merge.Metadata_METADATA_CONDITION_x[df_merge.Metadata_METADATA_CONDITION_x == 'DELETION'] = 1
df_merge.Metadata_METADATA_CONDITION_x[df_merge.Metadata_METADATA_CONDITION_x == 'CONTROL'] = 0

In [8]:
#select all feature columns
df_astro_feature=df_merge.loc[:,~df_merge.columns.str.startswith('Metadata')]

In [9]:
### logistic regression 
#define X and y
X=df_astro_feature.values
y=df_merge['Metadata_METADATA_CONDITION_x'].tolist()
y0=np.array(y)

#split training and testing 
#using GroupShuffleSplit to ensure the same patient won't appear in train and test set at the same time
gss = GroupShuffleSplit(n_splits=2, train_size=.8, random_state=42)
gss.get_n_splits()
for train_idx, test_idx in gss.split(X, y0, df_merge['Metadata_METADATA_SAMPLE_ID_x']):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y0[train_idx], y0[test_idx]

#fit the model
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
predictions = logisticRegr.predict(X_test)

#print acc score
score = logisticRegr.score(X_test, y_test)
score

0.65

In [10]:
#print feature importance values
astro_feature_weights = pd.DataFrame(logisticRegr.coef_[0], columns=['weight'], index=df_astro_feature.columns)
astro_feature_weights.reindex(astro_feature_weights['weight'].abs().sort_values(ascending=False).index).head(10)

,weight
Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel,-6.086396e-06
Cytoplasm_Correlation_Costes_Mito_DNA,-5.097806e-06
Cytoplasm_Correlation_Costes_Brightfield_DNA,-2.303927e-06
Nuclei_Correlation_Costes_RNA_ER,-2.753040e-07
Nuclei_Correlation_Costes_Mito_AGP,-2.057377e-07
Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,-1.362802e-07
Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,-1.247474e-07
Cells_Correlation_Costes_DNA_AGP,-8.821246e-08
Cells_Correlation_Costes_RNA_AGP,7.764760e-08
Nuclei_Correlation_Costes_Brightfield_RNA,-6.235065e-08


### mann-whitney-u-test

In [11]:
#mean profile based on patient ID and genotype
df_astro_mean= aggregate(
                    population_df=df_merge, 
                    strata=['Metadata_METADATA_SAMPLE_ID_x', 'Metadata_METADATA_CONDITION_x'], 
                    features='infer', 
                    operation='mean' 
                    )
df_astro_mean.head(2)

,Metadata_METADATA_SAMPLE_ID_x,Metadata_METADATA_CONDITION_x,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_1,...,Nuclei_Texture_InfoMeas1_AGP_5_00,Nuclei_Texture_InfoMeas1_Brightfield_5_00,Nuclei_Texture_InfoMeas1_DNA_5_02,Nuclei_Texture_InfoMeas1_RNA_5_00,Nuclei_Texture_InfoMeas2_DNA_5_02,Nuclei_Texture_SumVariance_Brightfield_20_00,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel
0,1,0,8.114575,1.952325,-0.201180,-3.579900,-2.087875,-5.515925,-2.864175,-0.851762,...,1.868082,-1.096847,0.109145,-0.623800,-0.830517,0.925197,49.005000,48.45225,3.048475,3427.2250
1,2,0,0.421964,-0.000109,0.278295,-0.101191,0.151920,-0.235490,-0.179907,0.053255,...,0.697913,-0.339472,-0.338145,0.408205,0.185370,-0.215720,9.940725,8.61310,3.630900,779.3275


In [12]:
#select all feature columns
df_astro_feature_u=df_astro_mean.loc[:,~df_astro_mean.columns.str.startswith('Metadata')]

In [13]:
#create feature list
feature_list=list(df_astro_feature_u.columns)

In [14]:
#create empty dataframe
df_p_values_feature = pd.DataFrame(index=[0],columns=feature_list)

In [15]:
#run mann-whitney-u test
list_p = []
list_u = []
for feat in feature_list:
    u, p = ss.mannwhitneyu(df_astro_mean.query('Metadata_METADATA_CONDITION_x == 1')[feat],          
           df_astro_mean.query('Metadata_METADATA_CONDITION_x == 0')[feat])
    list_p.append(p)
    list_u.append(u)

In [16]:
#store p value to each feature
df_p_values_feature.iloc[0] =list_p
df_p_values_feature

,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_3_3,Cells_AreaShape_Zernike_4_0,...,Nuclei_Texture_InfoMeas1_AGP_5_00,Nuclei_Texture_InfoMeas1_Brightfield_5_00,Nuclei_Texture_InfoMeas1_DNA_5_02,Nuclei_Texture_InfoMeas1_RNA_5_00,Nuclei_Texture_InfoMeas2_DNA_5_02,Nuclei_Texture_SumVariance_Brightfield_20_00,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel
0,0.271501,0.0164159,0.000831863,0.238425,0.00483002,0.336256,0.0550189,0.0464299,0.00686888,0.00244203,...,0.271501,0.00277418,1.03151e-05,0.0126356,2.12913e-05,0.00483002,0.0282364,0.430435,0.0244664,0.495887


In [17]:
#select features with p value smaller than .05
df_p_values_feature= df_p_values_feature.drop(df_p_values_feature.
                         columns[df_p_values_feature.iloc[-1,:]>0.001],axis=1)
df_p_values_feature.columns

Index(['Cells_AreaShape_EulerNumber', 'Cells_Correlation_Correlation_DNA_AGP',
       'Cells_Correlation_Correlation_DNA_RNA',
       'Cells_Correlation_Costes_Brightfield_ER',
       'Cells_Correlation_Costes_Brightfield_RNA',
       'Cells_Correlation_Costes_DNA_RNA', 'Cells_Correlation_K_AGP_ER',
       'Cells_Correlation_K_ER_AGP',
       'Cells_Correlation_Overlap_ER_Brightfield',
       'Cells_Correlation_Overlap_Mito_Brightfield',
       'Cells_Correlation_Overlap_RNA_Brightfield',
       'Cells_Intensity_MaxIntensityEdge_Brightfield',
       'Cells_RadialDistribution_RadialCV_AGP_3of4',
       'Cells_RadialDistribution_RadialCV_AGP_4of4',
       'Cells_RadialDistribution_RadialCV_Brightfield_4of4',
       'Cells_RadialDistribution_RadialCV_Mito_4of4',
       'Cells_RadialDistribution_RadialCV_RNA_4of4',
       'Cells_Texture_Correlation_DNA_5_00',
       'Cytoplasm_Correlation_Overlap_ER_Brightfield',
       'Cytoplasm_Correlation_RWC_ER_Mito',
       'Cytoplasm_Intensity_Lower